### Set access to google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/My\ Drive/CV_incubator/IncubatorCVProject

/content/drive/My Drive/CV_incubator/IncubatorCVProject


In [ ]:
!ls src

Basic_CNN_Architecture.py   data_loader.py	    main_two.py  train_valid.py
BasicCNN_withfeat_64x64.py  feature_engineering.py  __pycache__  utils.py


### Import libraries and setup paths


In [ ]:
from src.data_loader import SquarePadding
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor,Resize,Compose
from torch.utils.data.sampler import SubsetRandomSampler
import torch
import inspect
import numpy as np
import cv2
import os
import glob
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image
from src.feature_engineering import prepare_spectral_clustering_features,prepare_eigen_component_features

In [ ]:
# Path to image folder and label.csv
data_path = '../dog-breed-identification'

### Data loader




Map image path to label and label index

### Combining with engineered features

In [ ]:
from src.data_loader import load_dog_data
from sklearn.decomposition import PCA
from src.utils import PCA_images_list, unravel_image, ravel_image_vec, plot_image_grid
import numpy as np
import pickle
from src.feature_engineering import prepare_eigen_component_features
import torch
import pickle

In [ ]:
(image_list,label_list,label_dict) = load_dog_data(data_path,
              image_shape=(64,64),
              sample_rate=1, 
              simple=False)

In [ ]:
(eig_vals,eig_image_list) = PCA_images_list(image_list)

In [ ]:
pickle.dump( (eig_vals,eig_image_list), open( "eigvalsvecs.p", "wb" ) )

In [ ]:
pickle.dump(image_list, open( "image_list.p", "wb" ) )

In [ ]:
file = open('eigvalsvecs.p', 'rb')
eigen = pickle.load(file)
eig_vecs = eigen[1][-300:]

In [ ]:
def prepare_eigen_component_features(images_list,eig_vecs):
    '''
    Inputs:
        images_list: list of images (images are ndarrays, usually of 2 dimensions (e.g 128x128))
        eig_vecs: an 2d array of eigenvectors, where each eigenvector is a column
    Outputs:
        return: returns a feature matrix, where the rows of the feature matrix correspond to the 
        features of images. First image is the first row of the matrix, last image is the last row.
        The features are the components of the eigenvectors (which were passed in the input 'eig_vecs')
    '''
    #n_components = len(eig_vecs.shape)
    image_list = torch.Tensor(images_list)
    feat = torch.Tensor(eig_vecs)
    feat = torch.transpose(torch.flatten(feat,1),0,1)
    image_vec = torch.flatten(image_list,1)
    feature_matrix = image_vec@feat
        
    return feature_matrix


In [ ]:
image_vec = torch.flatten(torch.Tensor(image_list),1)
torch.transpose(image_vec,0,1).size()

torch.Size([4096, 10222])

In [ ]:
features = prepare_eigen_component_features(image_list,eig_vecs)
print(features.size())
pickle.dump( features, open( "features.p", "wb" ) )

torch.Size([10222, 300])


/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)


In [ ]:
file = open('features.p', 'rb')
features = pickle.load(file)
features.shape

torch.Size([10222, 300])

In [ ]:
train_feat = valid_feat = features

In [ ]:
torch.flatten(features).size()

torch.Size([3066600])